In [1]:
!pip install geemap

In [2]:
import os
import ee
import geemap
import pandas as pd
from datetime import datetime, timedelta
ee.Authenticate()
ee.Initialize()

In [3]:
pip install earthengine-api

In [4]:
Map = geemap.Map()
Map.setCenter(-123.3445771370641,48.43336760370573, 11)

In [5]:
### IMPORT HIGH WATER LINE ###

# imported from gee asssets

vic_coastline = ee.FeatureCollection("projects/ee-flickandzed/assets/victoriacoastline")
# Map.addLayer(vic_coastline)

Map.setCenter(-123.3445771370641,48.43336760370573, 11)

In [6]:
### BUFFER HIGH WATER LINE ###

roi = vic_coastline.map(lambda feature: feature.buffer(200))
roi = roi.union().geometry()
# Map.addLayer(roi)

Map.setCenter(-123.3445771370641,48.43336760370573, 11)

In [7]:
### DATE RANGE ###

# filtered for dates with low tide (under 1m) at 7:30PM UTM (noon pacific time) in excel

df = pd.read_csv('C:/Users/mcphersonf/Documents/work/oakbay2024.csv')

# print(df.info())

dates_list = df['dates_under_1m'].dropna().tolist()
dates_list = [date for date in dates_list if any(time in date for time in ['T18', 'T19', 'T20'])]
# print(dates_list)

cleaned_dates = [date.split('T')[0] for date in dates_list]
print(cleaned_dates)

['2024-03-13', '2024-03-14', '2024-03-28', '2024-03-29', '2024-03-30', '2024-04-10', '2024-04-11', '2024-04-12', '2024-04-13', '2024-04-25', '2024-04-26', '2024-04-27', '2024-04-28', '2024-05-10', '2024-05-11', '2024-05-12', '2024-05-25', '2024-05-26', '2024-05-27', '2024-05-28', '2024-06-08', '2024-06-09', '2024-06-10', '2024-06-11', '2024-06-23', '2024-06-24', '2024-06-25', '2024-06-26', '2024-07-07', '2024-07-08', '2024-07-09', '2024-07-10', '2024-07-23', '2024-07-24', '2024-07-25', '2024-08-06', '2024-08-07', '2024-08-21', '2024-08-22']


In [8]:
### DATE FILTER ###

date_filter = ee.Filter.date(cleaned_dates[0])
dates_added = [cleaned_dates[0]]

for date in cleaned_dates:
    if date not in dates_added:
        date_obj = datetime.strptime(date, "%Y-%m-%d")
        next_day = date_obj + timedelta(days=1)
        next_day_str = next_day.strftime("%Y-%m-%d")
        dates_added.append(next_day_str)

        while next_day_str in cleaned_dates:
            next_day = next_day + timedelta(days=1)
            next_day_str = next_day.strftime("%Y-%m-%d")
            dates_added.append(next_day_str)
        
        date_filter = ee.Filter.Or(
            ee.Filter.date(date, next_day_str), 
            date_filter
        )

# find number of images in that date range

# image_collection = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED").filterBounds(roi).filter(date_filter)
# image_count = image_collection.size().getInfo()
# print(f"Number of images in the filtered collection: {image_count}")
# print(image_collection.aggregate([ee.Reducer.toList(1), ee.List([ee.String('CLOUD_COVER_ASSESSMENT')])]))


In [9]:
### CHOOSE IMAGE ###

S2_image = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED").filterBounds(roi).filter(date_filter).sort("CLOUD_COVERAGE_ASSESSMENT").first()
trueColour = {'bands': ["B4", "B3", "B2"],'min': 0,'max': 2000}
Map.addLayer(S2_image.clip(roi), trueColour, 'S2 image')

Map.setCenter(-123.3445771370641,48.43336760370573, 11)
Map

Map(center=[48.43336760370573, -123.3445771370641], controls=(WidgetControl(options=['position', 'transparent_…

In [10]:
### Date the chosen image was taken ###

timestamp = S2_image.get('system:time_start')
date = ee.Date(timestamp).format("yyyy-MM-dd")
print(date.getInfo())

2024-07-23


In [11]:
### MANUAL TRAINING CLASSIFICATION ###

vegetation = ee.FeatureCollection("projects/ee-flickandzed/assets/vicvegetation")
water = ee.FeatureCollection("projects/ee-flickandzed/assets/vicwater")
infrastructure = ee.FeatureCollection("projects/ee-flickandzed/assets/vicinfrastructure")
bareground = ee.FeatureCollection("projects/ee-flickandzed/assets/vicbareground")

sub_S2 = S2_image.clip(roi)

newfc = water.merge(infrastructure).merge(vegetation).merge(bareground)
bands = ['B2', 'B3', 'B4', 'B8', 'B8A', 'B11', 'B12']

training = sub_S2.select(bands).sampleRegions(newfc, ['landcover'], 10)

classifier = ee.Classifier.smileRandomForest(150).train(training, 'landcover', bands)

classified = sub_S2.select(bands).classify(classifier)

dict = {"names": ["vegetation", "water", "infrastructure", "bareground"], "colours": ["#29d62c", "#5d52ff", "#d6d6d6", "#e6d53a"]}
Map.addLayer(classified, {'min': 0, 'max': 3, 'palette': dict['colours']}, "manual classification")

Map.setCenter(-123.3445771370641,48.43336760370573, 11)

In [12]:
### OTHER LANDCOVER DATASETS FROM GEE ###

nlcd = ee.Image("USGS/NLCD/NLCD2016").select("landcover").clip(S2_image.geometry())
dw = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1').filterDate('2024-01-01', '2024-12-31')
nlcd30m = ee.Image('USGS/NLCD_RELEASES/2020_REL/NALCMS').select("landcover").clip(S2_image.geometry())
Map.addLayer(nlcd, {}, "NLCD")
Map.addLayer(dw, {}, "DW")
Map.addLayer(nlcd30m, {}, "NLCD30M")

Map.setCenter(-123.3445771370641,48.43336760370573, 11)
# Map

In [13]:
### SPECTRAL INDICES ###

def add_indices(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('ndvi')
    ndbi = image.normalizedDifference(['B11', 'B8']).rename('ndbi')
    mndwi = image.normalizedDifference(['B3', 'B11']).rename('mndwi')
    bsi = image.expression(
        '((X + Y) - (A + B)) / ((X + Y) + (A + B))', {
            'X': image.select('B11'),  # swir1
            'Y': image.select('B4'),   # red
            'A': image.select('B8'),   # nir
            'B': image.select('B2')    # blue
        }).rename('bsi')
    return image.addBands([ndvi, ndbi, mndwi, bsi])

image_with_indices = add_indices(S2_image)

Map.addLayer(image_with_indices.clip(roi), trueColour, 'Image With Indices')

Map.setCenter(-123.3445771370641,48.43336760370573, 11)

In [14]:
### CEC’s North American Land Change Monitoring System (NALCMS) ###

# find the colour palette and order of classes

lc = ee.Image("projects/ee-flickandzed/assets/landcover-2020-classification")
dict = {"names": ["a", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m", "n"], "colours": ["#d60000", "#d6a300", "#6ad600", "#42d600", "#00d653", "#00d6c0", "#00bad6", "#001bd6", "#d400d6", "#7100d6", "#d6009c", "#d67760", "#55d651", "#7f6fd6", "#d68eb2"]}
Map.addLayer(lc, {'min': 0, 'max': 14, 'palette': dict['colours']}, "NALCMS classification")

In [15]:
### CLASSIFICATION USING DYNAMIC WORLD BANDS ###

In [16]:
### CLASSIFICAION USING ADDITIONAL SPECTRAL INDICES ###